# Feature engineering

As we have our that ready to train, we just need to try to adjust the cols that we have, so we can train our model better. We will start with a simple logistic regression training on the data to set a baselinemodel.

After this first superficial look, we are going select the most valuable features with sklearn sequential feature selection, using for this the Linear Support Vector Classification to get the first 40 atributes that are more significative.

In [ ]:
X = df.loc[:, df.columns != 'WIN']
y = df['WIN']

In [ ]:
X

,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,orb,...,opponent_drb%_max,opponent_trb%_max,opponent_ast%_max,opponent_stl%_max,opponent_blk%_max,opponent_tov%_max,opponent_usg%_max,opponent_ortg_max,opponent_drtg_max,opponent_Total
0,37.0,99.0,0.374,10.0,33.0,0.303,15.0,23.0,0.652,13.0,...,44.5,29.8,33.0,7.3,7.4,43.6,40.4,300.0,98.0,125
1,42.0,92.0,0.457,15.0,35.0,0.429,26.0,32.0,0.813,13.0,...,50.1,29.7,61.1,3.7,12.7,50.0,32.1,267.0,120.0,99
2,44.0,93.0,0.473,14.0,40.0,0.350,14.0,19.0,0.737,11.0,...,39.6,24.4,47.0,4.3,3.9,100.0,35.9,166.0,118.0,109
3,38.0,81.0,0.469,9.0,29.0,0.310,24.0,31.0,0.774,8.0,...,38.7,16.9,24.2,4.7,4.5,20.9,40.2,154.0,114.0,116
4,46.0,90.0,0.511,14.0,35.0,0.400,15.0,18.0,0.833,11.0,...,26.9,19.3,22.2,3.8,7.0,33.3,39.2,203.0,126.0,122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4979,34.0,85.0,0.400,15.0,38.0,0.395,14.0,19.0,0.737,11.0,...,29.1,20.0,30.2,6.1,10.3,42.9,36.3,133.0,112.0,107
4980,31.0,75.0,0.413,11.0,32.0,0.344,21.0,31.0,0.677,8.0,...,31.5,17.1,59.5,5.7,7.6,33.3,36.2,222.0,107.0,104
4981,41.0,88.0,0.466,9.0,40.0,0.225,13.0,15.0,0.867,4.0,...,84.8,42.4,100.0,2.3,12.4,45.0,94.4,300.0,112.0,94
4982,38.0,92.0,0.413,19.0,46.0,0.413,8.0,8.0,1.000,15.0,...,100.0,53.8,38.7,3.9,16.0,100.0,42.6,141.0,126.0,90


Info columns are the columns that present information of the next game to be predicted. So they will be used to group our features, so when performing rolling averages, will do it accordinly with these groups.


In [ ]:
info_cols = ['date', 'Team', 'season', 'home']
model_cols = X.columns[~X.columns.isin(info_cols)]	#only usefull for ML columns

In [ ]:
X[info_cols]

,date,Team,season,home
0,2020-12-22,GSW,2020-21,0
1,2020-12-22,BRK,2020-21,1
2,2020-12-22,LAC,2020-21,0
3,2020-12-22,LAL,2020-21,1
4,2020-12-23,MIL,2020-21,0
...,...,...,...,...
4979,2022-06-10,BOS,2021-22,1
4980,2022-06-13,BOS,2021-22,0
4981,2022-06-13,GSW,2021-22,1
4982,2022-06-16,GSW,2021-22,0


As we have many columns, we will run the feature selection method to extract only features that will be meaningful for training the model

We are going to use with this method , the TimeSeriesSplit method, because we dont want just to split totaly randonly the data for training and validation, as we want to use accordnly with the period of time. For example, I can use this to train against data per season.

In [ ]:
#minmaxscaler with traning data parameters
scaler = MinMaxScaler()
scaler.fit(X[model_cols])
#creating the pipeline
clf = make_pipeline(scaler, LinearSVC(random_state=42, C=1, tol=1e-5))
#creating the cross validation knowing that we gonna use different dates
split = TimeSeriesSplit(n_splits=5)
#choosing the number of features to select, it will be the first 25 features iteratively
sls = SequentialFeatureSelector(clf, n_features_to_select=25, cv=split, n_jobs=-1, direction='backward') # direction being backward to eliminates features instead of adding them to the setted value count ( takes a longer timer)


In [ ]:
#looking at our new data
X_transf = scaler.transform(X[model_cols])
X_transf = pd.DataFrame(X_transf, columns=model_cols)
X_transf.head()

,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,orb,...,opponent_drb%_max,opponent_trb%_max,opponent_ast%_max,opponent_stl%_max,opponent_blk%_max,opponent_tov%_max,opponent_usg%_max,opponent_ortg_max,opponent_drtg_max,opponent_Total
0,0.350,0.673469,0.236585,0.296296,0.433962,0.366057,0.333333,0.422222,0.497110,0.48,...,0.330519,0.227723,0.202381,0.073,0.074,0.408805,0.234917,1.000000,0.168831,0.611765
1,0.475,0.530612,0.439024,0.481481,0.471698,0.575707,0.638889,0.622222,0.729769,0.48,...,0.398070,0.226623,0.536905,0.037,0.127,0.475891,0.128370,0.829897,0.454545,0.305882
2,0.525,0.551020,0.478049,0.444444,0.566038,0.444260,0.305556,0.333333,0.619942,0.40,...,0.271411,0.168317,0.369048,0.043,0.039,1.000000,0.177150,0.309278,0.428571,0.423529
3,0.375,0.306122,0.468293,0.259259,0.358491,0.377704,0.583333,0.600000,0.673410,0.28,...,0.260555,0.085809,0.097619,0.047,0.045,0.170860,0.232349,0.247423,0.376623,0.505882
4,0.575,0.489796,0.570732,0.444444,0.471698,0.527454,0.333333,0.311111,0.758671,0.40,...,0.118215,0.112211,0.073810,0.038,0.070,0.300839,0.219512,0.500000,0.532468,0.576471


In [ ]:
#sls.fit(X[model_cols], y)

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None),
                          direction='backward',
                          estimator=Pipeline(steps=[('minmaxscaler',
                                                     MinMaxScaler()),
                                                    ('linearsvc',
                                                     LinearSVC(C=1,
                                                               random_state=42,
                                                               tol=1e-05))]),
                          n_features_to_select=25, n_jobs=-1)

In [ ]:
# getting the selected features
predictors = list(model_cols[sls.get_support()]) # the list will contain only columns that get support say it is true, from model columns
#saving our actual predictors list
with open('predictors.txt', 'w') as f:
    f.write(str(predictors))
predictors

['fga',
 'pts',
 'ts%',
 'ortg',
 'drtg',
 'pf_max',
 'pts_max',
 'Total',
 'opponent_fg',
 'opponent_3pa',
 'opponent_pts',
 'opponent_ts%',
 'opponent_orb%',
 'opponent_tov%',
 'opponent_ortg',
 'opponent_drtg',
 'opponent_3p_max',
 'opponent_ft_max',
 'opponent_stl_max',
 'opponent_pf_max',
 'opponent_3par_max',
 'opponent_tov%_max',
 'opponent_usg%_max',
 'opponent_ortg_max',
 'opponent_Total']

In [ ]:
#getting our predictions columns back
with open('predictors.txt', 'r') as f:
    predictors = f.read()
predictors = predictors.replace('[', '').replace(']', '').replace("'", '').split(', ')
predictors

['fga',
 'pts',
 'ts%',
 'ortg',
 'drtg',
 'pf_max',
 'pts_max',
 'Total',
 'opponent_fg',
 'opponent_3pa',
 'opponent_pts',
 'opponent_ts%',
 'opponent_orb%',
 'opponent_tov%',
 'opponent_ortg',
 'opponent_drtg',
 'opponent_3p_max',
 'opponent_ft_max',
 'opponent_stl_max',
 'opponent_pf_max',
 'opponent_3par_max',
 'opponent_tov%_max',
 'opponent_usg%_max',
 'opponent_ortg_max',
 'opponent_Total']

In [ ]:
scaler.fit_transform(X[model_cols])

array([[0.35      , 0.67346939, 0.23658537, ..., 1.        , 0.16883117,
        0.61176471],
       [0.475     , 0.53061224, 0.43902439, ..., 0.82989691, 0.45454545,
        0.30588235],
       [0.525     , 0.55102041, 0.47804878, ..., 0.30927835, 0.42857143,
        0.42352941],
       ...,
       [0.45      , 0.44897959, 0.46097561, ..., 1.        , 0.35064935,
        0.24705882],
       [0.375     , 0.53061224, 0.33170732, ..., 0.18041237, 0.53246753,
        0.2       ],
       [0.275     , 0.28571429, 0.36097561, ..., 0.41237113, 0.33766234,
        0.35294118]])